In [266]:
import pandas as pd
import string
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import unicodedata
from sklearn.linear_model import RidgeCV
import numpy as np
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import shap
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from scikeras.wrappers import KerasRegressor
import xgboost as xgb
import tensorflow as tf
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [267]:
X_train = pd.read_csv('/Users/emilfalk/Desktop/X_train.csv')
X_test = pd.read_csv('/Users/emilfalk/Desktop/X_test.csv')
y_train = pd.read_csv('/Users/emilfalk/Desktop/y_train.csv')
y_test = pd.read_csv('/Users/emilfalk/Desktop/y_test.csv')

In [268]:
X_train = X_train.drop(columns=['Player']) 
X_test = X_test.drop(columns=['Player']) 

In [269]:
X_train_basketball = X_train.drop(['years_with_team', 'draft_pos', 'all_star', 'num_teams', 'ig_followers', 'resigned', 'Agent'], axis=1)
X_test_basketball = X_test.drop(['years_with_team', 'draft_pos', 'all_star', 'num_teams', 'ig_followers', 'resigned', 'Agent'], axis=1)

In [270]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)
basketball_scaler = StandardScaler()
X_train_basketball_scaled = basketball_scaler.fit_transform(X_train_basketball) 
X_test_basketball_scaled = basketball_scaler.transform(X_test_basketball)

In [271]:
X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)
X_train_basketball_cnn = X_train_basketball_scaled.reshape(X_train_basketball_scaled.shape[0], X_train_basketball_scaled.shape[1], 1)
X_test_basketball_cnn = X_test_basketball_scaled.reshape(X_test_basketball_scaled.shape[0], X_test_basketball_scaled.shape[1], 1)
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [272]:
def create_optimized_1d_cnn(input_dim=28, output_dim=1, learning_rate=0.01, 
                            filters_1=16, filters_2=8, dense_units=50, 
                            dropout_rate_1=0.3, dropout_rate_2=0.3, dropout_rate_3=0.4):
    model = models.Sequential()
    
    # Fully connected layer to increase dimension
    model.add(layers.Dense(128, input_shape=(input_dim,), activation='relu'))
    
    # Reshape to (16, 8) for Conv1D input
    model.add(layers.Reshape((16, 8)))
    
    # First Conv1D layer with batch normalization and dropout
    model.add(layers.Conv1D(filters=filters_1, kernel_size=3, activation='leaky_relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate_1))
    
    # Second Conv1D layer with batch normalization and dropout
    model.add(layers.Conv1D(filters=filters_2, kernel_size=3, activation='leaky_relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate_2))
    
    # Flatten the output
    model.add(layers.Flatten())
    
    # Dense layer for regression
    model.add(layers.Dense(dense_units, activation='relu'))
    model.add(layers.Dropout(dropout_rate_3))
    
    # Output layer for regression
    model.add(layers.Dense(output_dim))
    
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    
    return model

In [264]:
model = create_optimized_1d_cnn(input_dim=X_train_basketball_cnn.shape[1])
model.fit(X_train_basketball_cnn, y_train_log, epochs=450, batch_size=16, validation_split=0.2)

Epoch 1/450


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 140.4614 - val_loss: 146.5183
Epoch 2/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.6564 - val_loss: 88.3968
Epoch 3/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21.2699 - val_loss: 57.8542
Epoch 4/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17.1387 - val_loss: 19.9852
Epoch 5/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16.1017 - val_loss: 16.2200
Epoch 6/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.5608 - val_loss: 12.0452
Epoch 7/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14.4628 - val_loss: 4.4628
Epoch 8/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11.8685 - val_loss: 3.7767
Epoch 9/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11.8752 - val_loss: 5.0471
Epoch 10/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11.0022 - val_loss: 1.6065
Epoch 11/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.6561 - val_loss: 1.5211
Epoch 12/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - los

In [265]:
y_pred_log = model.predict(X_test_basketball_cnn).flatten()

y_pred_actual = np.exp(y_pred_log)
y_test_actual = np.exp(y_test_log)

mae = mean_absolute_error(y_test_actual, y_pred_actual)
r2 = r2_score(y_test_actual, y_pred_actual)

print(f"MAE (actual values): {mae}")
print(f"R² (actual values): {r2}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
MAE (actual values): 3691126.5203651683
R² (actual values): 0.6120450496673584


In [274]:
model = create_optimized_1d_cnn(input_dim=X_train_cnn.shape[1])
model.fit(X_train_cnn, y_train_log, epochs=450, batch_size=16, validation_split=0.2)

Epoch 1/450


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 135.3037 - val_loss: 135.3493
Epoch 2/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24.2867 - val_loss: 65.8011
Epoch 3/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21.6815 - val_loss: 34.3219
Epoch 4/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19.6956 - val_loss: 20.5516
Epoch 5/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18.0246 - val_loss: 13.3516
Epoch 6/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.3075 - val_loss: 8.1682
Epoch 7/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.3436 - val_loss: 6.5744
Epoch 8/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.2020 - val_loss: 5.0171
Epoch 9/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.0638 - val_loss: 1.6582
Epoch 10/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12.7456 - val_loss: 1.4635
Epoch 11/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11.1342 - val_loss: 2.2538
Epoch 12/450
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - lo

In [276]:
y_pred_log = model.predict(X_test_cnn).flatten()

y_pred_actual = np.exp(y_pred_log)
y_test_actual = np.exp(y_test_log)

bb_mae = mean_absolute_error(y_test_actual, y_pred_actual)
bb_r2 = r2_score(y_test_actual, y_pred_actual)

print(f"MAE (actual values): {mae}")
print(f"R² (actual values): {r2}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
MAE (actual values): 3578443.991046351
R² (actual values): 0.6886184811592102


In [278]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

def create_optimized_1d_cnn(input_dim=28, output_dim=1, learning_rate=0.01, 
                            filters_1=16, filters_2=8, dense_units=50, 
                            dropout_rate_1=0.3, dropout_rate_2=0.3, dropout_rate_3=0.4):
    model = models.Sequential()
    model.add(layers.Dense(128, input_shape=(input_dim,), activation='relu'))
    model.add(layers.Reshape((16, 8)))  

    model.add(layers.Conv1D(filters=filters_1, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate_1))

    model.add(layers.Conv1D(filters=filters_2, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate_2))

    model.add(layers.Flatten())
    model.add(layers.Dense(dense_units, activation='relu'))
    model.add(layers.Dropout(dropout_rate_3))
    model.add(layers.Dense(output_dim))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Define early stopping
early_stop = EarlyStopping(patience=200, restore_best_weights=True)

# Set up K-Fold CV
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Empty lists to store results
maes_basketball = []
r2s_basketball = []

maes_full = []
r2s_full = []

for train_idx, test_idx in kf.split(X_train_basketball_cnn):
    # --- Basketball Features Model ---
    X_train_fold = X_train_basketball_cnn[train_idx]
    X_test_fold = X_train_basketball_cnn[test_idx]
    y_train_fold = y_train_log.iloc[train_idx]
    y_test_fold = y_train_log.iloc[test_idx]

    model = create_optimized_1d_cnn(input_dim=X_train_basketball_cnn.shape[1])
    model.fit(X_train_fold, y_train_fold, epochs=450, batch_size=16, 
              validation_split=0.2, callbacks=[early_stop], verbose=0)

    y_pred_log = model.predict(X_test_fold).flatten()
    y_pred_actual = np.exp(y_pred_log)
    y_test_actual = np.exp(y_test_fold)

    mae = mean_absolute_error(y_test_actual, y_pred_actual)
    r2 = r2_score(y_test_actual, y_pred_actual)

    maes_basketball.append(mae)
    r2s_basketball.append(r2)

for train_idx, test_idx in kf.split(X_train_cnn):
    # --- Full Features Model ---
    X_train_fold = X_train_cnn[train_idx]
    X_test_fold = X_train_cnn[test_idx]
    y_train_fold = y_train_log.iloc[train_idx]
    y_test_fold = y_train_log.iloc[test_idx]

    model = create_optimized_1d_cnn(input_dim=X_train_cnn.shape[1])
    model.fit(X_train_fold, y_train_fold, epochs=450, batch_size=16, 
              validation_split=0.2, callbacks=[early_stop], verbose=0)

    y_pred_log = model.predict(X_test_fold).flatten()
    y_pred_actual = np.exp(y_pred_log)
    y_test_actual = np.exp(y_test_fold)

    mae = mean_absolute_error(y_test_actual, y_pred_actual)
    r2 = r2_score(y_test_actual, y_pred_actual)

    maes_full.append(mae)
    r2s_full.append(r2)

# Print final results
print("\n=== Basketball Features ===")
print(f"MAE: Mean = {np.mean(maes_basketball):.2f}, Std = {np.std(maes_basketball, ddof=1):.2f}")
print(f"R²:  Mean = {np.mean(r2s_basketball):.3f}, Std = {np.std(r2s_basketball, ddof=1):.3f}")

print("\n=== Full Features ===")
print(f"MAE: Mean = {np.mean(maes_full):.2f}, Std = {np.std(maes_full, ddof=1):.2f}")
print(f"R²:  Mean = {np.mean(r2s_full):.3f}, Std = {np.std(r2s_full, ddof=1):.3f}")

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

=== Basketball Features ===
MAE: Mean = 3723348.51, Std = 473381.83
R²:  Mean = 0.630, Std = 0.108

=== Full Features ===
MAE: Mean = 3576049.80, Std = 548020.24
R²:  Mean = 0.595, Std = 0.185


In [279]:
print("\n=== Basketball Features ===")
print(f"MAE: Mean = {np.mean(maes_basketball):.2f}, Std = {np.std(maes_basketball, ddof=1):.2f}")
print(f"R²:  Mean = {np.mean(r2s_basketball):.4f}, Std = {np.std(r2s_basketball, ddof=1):.3f}")

print("\n=== Full Features ===")
print(f"MAE: Mean = {np.mean(maes_full):.2f}, Std = {np.std(maes_full, ddof=1):.2f}")
print(f"R²:  Mean = {np.mean(r2s_full):.4f}, Std = {np.std(r2s_full, ddof=1):.3f}")


=== Basketball Features ===
MAE: Mean = 3723348.51, Std = 473381.83
R²:  Mean = 0.6296, Std = 0.108

=== Full Features ===
MAE: Mean = 3576049.80, Std = 548020.24
R²:  Mean = 0.5945, Std = 0.185
